In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Large/'

file_names = ['berlin52.npz', 'brazil58.npz', 'st70.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Expected Constraint', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 20

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=8)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=1)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,1121.0,12855,0,12855.0
1,3249,5192.0,50338,0,50338.0
2,4761,76.0,1244,0,1244.0
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,1121.0,19009,0,19009.0
1,3249,5192.0,73121,0,73121.0
2,4761,76.0,2074,0,2074.0
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,1121.0,10654,0,10654.0
1,3249,5192.0,45310,0,45310.0


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,Energy (minimisation) 10,Energy (minimisation) 11,Energy (minimisation) 12,Energy (minimisation) 13,Energy (minimisation) 14,Energy (minimisation) 15,Energy (minimisation) 16,Energy (minimisation) 17,Energy (minimisation) 18,Energy (minimisation) 19
0,10654.0,10735.0,10793.0,11430.0,10978.0,10831.0,10797.0,11010.0,10905.0,12189.0,10656.0,11787.0,11327.0,11766.0,11055.0,10820.0,10657.0,11289.0,11006.0,10412.0
1,45310.0,40602.0,42920.0,42683.0,42970.0,44739.0,42157.0,45539.0,43761.0,44723.0,47840.0,45779.0,42323.0,44549.0,41485.0,41235.0,39591.0,45703.0,40225.0,48901.0
2,1580.0,1381.0,1426.0,1110.0,1815.0,1361.0,1418.0,1375.0,1214.0,1336.0,1509.0,1403.0,1530.0,1156.0,1494.0,1140.0,1439.0,1446.0,1621.0,1329.0


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,Broken Constraints 10,Broken Constraints 11,Broken Constraints 12,Broken Constraints 13,Broken Constraints 14,Broken Constraints 15,Broken Constraints 16,Broken Constraints 17,Broken Constraints 18,Broken Constraints 19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,Feasible 10,Feasible 11,Feasible 12,Feasible 13,Feasible 14,Feasible 15,Feasible 16,Feasible 17,Feasible 18,Feasible 19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,False,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.000000,1.000000,13400.350000,546.366493
1,20.000000,1.000000,48453.200000,3465.607103
2,18.000000,0.900000,1321.100000,76.672166
Total,58.000000,2.900000,63174.650000,4088.645762
Mean,19.333333,0.966667,21058.216667,1362.881921
SD,1.154701,0.057735,24481.439991,1836.094519
,Feasible,Feasibility rate,Energy mean,Energy SD
0,19.0,0.95,19032.400000,946.002693
1,20.0,1.00,73609.400000,5413.353727
2,18.0,0.90,1909.350000,95.355995


# Save results

In [13]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Large/'
broken_constraints_greedy.to_pickle(data_folder + 'expected_tspL_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'expected_tspL_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'expected_tspL_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'expected_tspL_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'expected_tspL_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'expected_tspL_tabu_feasible.pkl')